<a href="https://colab.research.google.com/github/ScriptSherpa/TrackCount360-YOLO-Based-Object-Tracking-Directional-Counting-System/blob/main/detect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Install necessary packages
!pip install ultralytics opencv-python-headless

# Import required libraries
import cv2
from ultralytics import YOLO
from google.colab import files
import numpy as np
from PIL import Image

# Upload your video file
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# Load the YOLOv8 model
model = YOLO("yolov8l.pt")  # You can replace with yolov8m.pt or yolov8n.pt based on your preference

# Open the video file
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print("Error: Couldn't open video.")
    exit()

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Prepare output video writer
output_path = "/content/output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

# Process each frame
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Perform object detection
    results = model(frame)
    annotated_frame = np.array(results[0].plot())

    # Write the frame to the output video
    out.write(annotated_frame)

    # Display the frame (optional)
    Image.fromarray(annotated_frame).show()

# Release resources
cap.release()
out.release()

# Provide the output video for download
files.download(output_path)


Saving cars.mp4 to cars (1).mp4


100%|██████████| 83.7M/83.7M [00:00<00:00, 215MB/s]



0: 384x640 1 person, 14 cars, 1 truck, 1 traffic light, 1 stop sign, 1820.0ms
Speed: 19.1ms preprocess, 1820.0ms inference, 29.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 14 cars, 1 truck, 1 traffic light, 1 stop sign, 1718.2ms
Speed: 6.2ms preprocess, 1718.2ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 16 cars, 1 truck, 1 traffic light, 1 stop sign, 1597.4ms
Speed: 3.2ms preprocess, 1597.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 16 cars, 1 traffic light, 1 stop sign, 2003.4ms
Speed: 2.9ms preprocess, 2003.4ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 13 cars, 2 trucks, 1 traffic light, 2042.9ms
Speed: 7.8ms preprocess, 2042.9ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 12 cars, 2 trucks, 1 traffic light, 1605.2ms
Speed: 5.0ms preprocess, 1605.2ms inference, 1.4ms postprocess per image at shape 

KeyboardInterrupt: 

In [ ]:
# Install necessary packages
!pip install ultralytics opencv-python-headless opencv-contrib-python-headless

# Import required libraries
from ultralytics import YOLO
import cv2
import numpy as np
from sort import Sort

# Initialize video capture
cap = cv2.VideoCapture("../Videos/people.mp4")

# Load the YOLOv8 model
model = YOLO("../Yolo-Weights/yolov8l.pt")

# Define class names
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"]

# Initialize the SORT tracker
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)

# Define line limits for counting
limitsUp = [103, 161, 296, 161]
limitsDown = [527, 489, 735, 489]

# Initialize counters
totalCountUp = []
totalCountDown = []

while True:
    success, img = cap.read()
    if not success:
        break

    # Perform inference
    results = model(img, stream=True)
    detections = np.empty((0, 5))

    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            w, h = x2 - x1, y2 - y1

            # Confidence
            conf = box.conf[0].item()
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]

            if currentClass == "person" and conf > 0.3:
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))

    # Update tracker
    resultsTracker = tracker.update(detections)

    # Draw lines
    cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 0, 255), 5)
    cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 0, 255), 5)

    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        w, h = x2 - x1, y2 - y1

        # Draw bounding box and ID
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
        cv2.putText(img, f'ID: {int(id)}', (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 3)

        # Center point
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), -1)

        # Count vehicles crossing the lines
        if limitsUp[0] < cx < limitsUp[2] and limitsUp[1] - 15 < cy < limitsUp[1] + 15:
            if id not in totalCountUp:
                totalCountUp.append(id)
                cv2.line(img, (limitsUp[0], limitsUp[1]), (limitsUp[2], limitsUp[3]), (0, 255, 0), 5)

        if limitsDown[0] < cx < limitsDown[2] and limitsDown[1] - 15 < cy < limitsDown[1] + 15:
            if id not in totalCountDown:
                totalCountDown.append(id)
                cv2.line(img, (limitsDown[0], limitsDown[1]), (limitsDown[2], limitsDown[3]), (0, 255, 0), 5)

    # Display counts
    cv2.putText(img, f'Up: {len(totalCountUp)}', (50, 50), cv2.FONT_HERSHEY_PLAIN, 5, (0, 255, 0), 3)
    cv2.putText(img, f'Down: {len(totalCountDown)}', (50, 100), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 255), 3)

    # Show the image
    cv2_imshow(img)
    cv2.waitKey(1)
